In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [1]:
#First, let's see which year we have the most data on, so we can compare those instead of the whole 2016-2024 timeframe
df = pd.read_csv("final_df.csv")

#we group by year and count the number of articles
articles_per_year = df['year'].value_counts().sort_index()

print(articles_per_year)


2016    261
2017    231
2018    432
2019    905
2020    279
2021    236
2022    113
2023     64
2024     11
Name: year, dtype: int64


In [9]:
#then we re-use the function from the main scraping but only scrape articles, and from the main page

#this code was partially generated with the help of ChatGPT-4o-mini 
def scrape_pages(start_page, end_page):
    base_url = 'https://www.thejournal.ie/'
    all_article_urls = set()
    article_data = []

    for page_num in range(start_page, end_page + 1):
        url = base_url if page_num == 1 else f'{base_url}page/{page_num}/'
        print(f"Fetching page: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch {url}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', class_='link-overlay-redesign')
        for link in links:
            all_article_urls.add(link['href'])

    print(f"Total articles found: {len(all_article_urls)}")

    for article_url in all_article_urls:
        print(f"Scraping article: {article_url}")
        try:
            article_response = requests.get(article_url)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            article_div = article_soup.find('div', class_='article-content-redesign')
            article_text = ''
            if article_div:
                paragraphs = article_div.find_all('p')
                article_text = '\n'.join(p.get_text(strip=True) for p in paragraphs)
            else:
                print(f"No article text found for {article_url}")

            creation_date_div = article_soup.find('div', class_='metadata-csp-date')
            article_date = creation_date_div.get_text(strip=True) if creation_date_div else None

            updated_span = article_soup.find('span', class_='article-updated-time-redesign')
            article_updated_date = updated_span.get_text(strip=True) if updated_span else None

            article_data.append({
                'article_url': article_url,
                'article_text': article_text,
                'article_date_created': article_date,
                'article_date_updated': article_updated_date
            })

            time.sleep(1)  
        except Exception as e:
            print(f"Error scraping {article_url}: {e}")

    articles_df = pd.DataFrame(article_data)
    articles_df.to_csv(f'articles_{start_page}_{end_page}.csv', index=False)
    print(f"Saved articles_{start_page}_{end_page}.csv")


In [12]:
#and we scrape about 900 articles from 2019 
#(the page number was selected after manually looking at which pages were 2019 and counting 900 articles across all the pages)
scrape_pages(2400, 2422)

Fetching page: https://www.thejournal.ie/page/2400/
Fetching page: https://www.thejournal.ie/page/2401/
Fetching page: https://www.thejournal.ie/page/2402/
Fetching page: https://www.thejournal.ie/page/2403/
Fetching page: https://www.thejournal.ie/page/2404/
Fetching page: https://www.thejournal.ie/page/2405/
Fetching page: https://www.thejournal.ie/page/2406/
Fetching page: https://www.thejournal.ie/page/2407/
Fetching page: https://www.thejournal.ie/page/2408/
Fetching page: https://www.thejournal.ie/page/2409/
Fetching page: https://www.thejournal.ie/page/2410/
Fetching page: https://www.thejournal.ie/page/2411/
Fetching page: https://www.thejournal.ie/page/2412/
Fetching page: https://www.thejournal.ie/page/2413/
Fetching page: https://www.thejournal.ie/page/2414/
Fetching page: https://www.thejournal.ie/page/2415/
Fetching page: https://www.thejournal.ie/page/2416/
Fetching page: https://www.thejournal.ie/page/2417/
Fetching page: https://www.thejournal.ie/page/2418/
Fetching pag